# Front matter

_Important_: On first run, set CACHING to True

In [ ]:
CACHING = False
EXPERIMENTAL = False

## Set path

In [ ]:
PATH_YIELD_ENGINE = 'src'

def set_paths() -> None:
    """
    set correct working directory and python path when started from within PyCharm
    """
    import sys
    import os
    
    if 'cwd' not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir)
        os.chdir(cwd)
    
    print(f"working dir is '{os.getcwd()}'")
                             
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    
    print(f"added `{sys.path[0]}` to python paths")

set_paths()

## Imports

In [ ]:
import logging
import os
import pickle

import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRegressor
from matplotlib import cm
from matplotlib.pyplot import figure
from sklearn.ensemble import RandomForestRegressor

import tests
from tests.model import make_simple_transformer
from tests.paths import TEST_DATA_CSV
from yieldengine import Sample
from yieldengine.dendrogram import DendrogramDrawer
from yieldengine.dendrogram.style import DendrogramHeatmapStyle, DendrogramLineStyle
from yieldengine.preprocessing.impute import SimpleImputerDF
from yieldengine.preprocessing.selection import BorutaDF
from yieldengine.model.inspection import ModelInspector
from yieldengine.model.prediction import ModelFitCV
from yieldengine.df.pipeline import PipelineDF
from yieldengine.model.selection import ModelPipelineDF, ModelGrid, BaseLearnerRanker
from yieldengine.model.validation import CircularCV


In [ ]:
%matplotlib inline

In [ ]:
PATH_TMP = 'tmp'
MI_PKL = os.path.join(PATH_TMP, 'model_inspector.pkl')

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

## Load the data file

In [ ]:
inputfile_config = tests.read_test_config(section="inputfile")

batch_df = pd.read_csv(
    filepath_or_buffer=TEST_DATA_CSV,
    delimiter=inputfile_config["delimiter"],
    header=inputfile_config["header"],
    decimal=inputfile_config["decimal"],
)

batch_df = batch_df.drop(columns=["Date", "Batch Id"])

# replace values of +/- infinite with n/a, then drop all n/a columns:
batch_df = batch_df.replace([np.inf, -np.inf], np.nan).dropna(
    axis=1, how="all"
)
batch_df.head()

In [ ]:
# define a Sample based on the test batch_file
sample = Sample(observations=batch_df, target="Yield")
sample

In [ ]:
# define the circular cross validator with 10 folds
circular_cv = CircularCV(test_ratio=0.2, num_splits=10)

circular_cv

In [ ]:
if not CACHING:
    boruta_selector = PipelineDF(steps=[
        (
            'preprocess', 
            make_simple_transformer(
                impute_median_columns=sample.features_by_type(Sample.DTYPE_NUMERICAL).columns,
                one_hot_encode_columns=sample.features_by_type(Sample.DTYPE_OBJECT).columns,
            )
        ),
        (
            'boruta', 
            BorutaDF(
                estimator=RandomForestRegressor(n_jobs=4),
                max_iter=100,
                n_estimators='auto', 
                verbose=2, 
                random_state=42
            )
        )
    ])
    
    feature_selection = boruta_selector.fit_transform(
            sample.features, 
            sample.target
        )
    
    selected_features = boruta_selector.features_original

    sample_post_boruta = sample.select_features(selected_features)
    
    selected_features.to_frame()

In [ ]:
# define a transformer step based on the sample
if not CACHING:
    preprocessor = make_simple_transformer(
            impute_median_columns=sample_post_boruta.features_by_type(Sample.DTYPE_NUMERICAL).columns,
            one_hot_encode_columns=sample_post_boruta.features_by_type(Sample.DTYPE_OBJECT).columns,
    )
    preprocessor

In [ ]:
if not CACHING:
    # define a ModelPipelineDF with a preprocessing pipeline
    lgbm = ModelGrid(
                model=ModelPipelineDF(
                    preprocessing=preprocessor, estimator=LGBMRegressor()
                ),
                learner_parameters={
                    "max_depth": [5, 10],
                    "min_split_gain": [0.1, 0.2],
                    "num_leaves": [50, 100, 200],
                    "random_state": [42],
                },
    )
    lgbm

In [ ]:
if not CACHING:
    # define a BaseLearnerRanker
    model_ranker: BaseLearnerRanker = BaseLearnerRanker(
            grids=[lgbm], cv=circular_cv, scoring="r2"
        )

    # run the BaseLearnerRanker to retrieve a ranking
    model_ranking = model_ranker.run(sample=sample_post_boruta)
    # noinspection PyStatementEffect
else:
    model_ranking = None
    
model_ranking

In [ ]:
if not CACHING:
    # retrieve the best model
    best_model = model_ranking[0]
    # noinspection PyStatementEffect
    best_model
else:
    best_model = None

In [ ]:
if not CACHING:
    # defina a Predictor CV
    mf = ModelFitCV(
        model=best_model.model,
        cv=circular_cv,
        sample=sample
    )
    # define a ModelInspector
    mi = ModelInspector(model_fit=mf)

    mi.feature_dependency_matrix()

    with open(MI_PKL, 'wb') as f:
        pickle.dump(mi, f)
else:
    with open(MI_PKL, 'rb') as f:
        mi = pickle.load(f)

mi

In [ ]:
# retrieve a the shap_values
mi.shap_values().head()

In [ ]:
mi.feature_dependency_matrix().head()

In [ ]:
if EXPERIMENTAL:
    mi.plot_feature_dendrogram_scipy()

In [ ]:
plasma=cm.get_cmap(name="plasma", lut=256)

In [ ]:
fig = figure(figsize=(8,16))
ax = fig.add_subplot(111)
linkage_tree = mi.cluster_dependent_features()

drawer = DendrogramDrawer(
    linkage_tree=linkage_tree,
    style=DendrogramHeatmapStyle(ax=ax),
    title='Feature clusters'
)


drawer.draw()

In [ ]:
linkage_tree.n_leaves

In [ ]:
fig = figure(figsize=(8,16))
ax = fig.add_subplot(111)
linkage_tree = mi.cluster_dependent_features()

drawer = DendrogramDrawer(
    linkage_tree=linkage_tree,
    style=DendrogramLineStyle(ax=ax),
    title='Feature clusters'
)

drawer.draw()